# Project: Identify Customer Segments

In this project, you will apply unsupervised learning techniques to identify segments of the population that form the core customer base for a mail-order sales company in Germany. These segments can then be used to direct marketing campaigns towards audiences that will have the highest expected rate of returns. The data that you will use has been provided by our partners at Bertelsmann Arvato Analytics, and represents a real-life data science task.

This notebook will help you complete this task by providing a framework within which you will perform your analysis steps. In each step of the project, you will see some text describing the subtask that you will perform, followed by one or more code cells for you to complete your work. **Feel free to add additional code and markdown cells as you go along so that you can explore everything in precise chunks.** The code cells provided in the base template will outline only the major tasks, and will usually not be enough to cover all of the minor tasks that comprise it.

It should be noted that while there will be precise guidelines on how you should handle certain tasks in the project, there will also be places where an exact specification is not provided. **There will be times in the project where you will need to make and justify your own decisions on how to treat the data.** These are places where there may not be only one way to handle the data. In real-life tasks, there may be many valid ways to approach an analysis task. One of the most important things you can do is clearly document your approach so that other scientists can understand the decisions you've made.

At the end of most sections, there will be a Markdown cell labeled **Discussion**. In these cells, you will report your findings for the completed section, as well as document the decisions that you made in your approach to each subtask. **Your project will be evaluated not just on the code used to complete the tasks outlined, but also your communication about your observations and conclusions at each stage.**

In [1]:
# import libraries here; add more as necessary
#upgrading plotly version
!pip install plotly==4.8.1
#upgrading sklearn version
!pip install scikit-learn==0.23.1
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

# magic word for producing visualizations in notebook
%matplotlib inline

#installing pandas profiling package
# # !pip install pandas-profiling
# from pandas_profiling import ProfileReport
'''
Import note: The classroom currently uses sklearn version 0.19.
If you need to use an imputer, it is available in sklearn.preprocessing.Imputer,
instead of sklearn.impute as in newer versions of sklearn.
'''

'\nImport note: The classroom currently uses sklearn version 0.19.\nIf you need to use an imputer, it is available in sklearn.preprocessing.Imputer,\ninstead of sklearn.impute as in newer versions of sklearn.\n'

### Step 0: Load the Data

There are four files associated with this project (not including this one):

- `Udacity_AZDIAS_Subset.csv`: Demographics data for the general population of Germany; 891211 persons (rows) x 85 features (columns).
- `Udacity_CUSTOMERS_Subset.csv`: Demographics data for customers of a mail-order company; 191652 persons (rows) x 85 features (columns).
- `Data_Dictionary.md`: Detailed information file about the features in the provided datasets.
- `AZDIAS_Feature_Summary.csv`: Summary of feature attributes for demographics data; 85 features (rows) x 4 columns

Each row of the demographics files represents a single person, but also includes information outside of individuals, including information about their household, building, and neighborhood. You will use this information to cluster the general population into groups with similar demographic properties. Then, you will see how the people in the customers dataset fit into those created clusters. The hope here is that certain clusters are over-represented in the customers data, as compared to the general population; those over-represented clusters will be assumed to be part of the core userbase. This information can then be used for further applications, such as targeting for a marketing campaign.

To start off with, load in the demographics data for the general population into a pandas DataFrame, and do the same for the feature attributes summary. Note for all of the `.csv` data files in this project: they're semicolon (`;`) delimited, so you'll need an additional argument in your [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) call to read in the data properly. Also, considering the size of the main dataset, it may take some time for it to load completely.

Once the dataset is loaded, it's recommended that you take a little bit of time just browsing the general structure of the dataset and feature summary file. You'll be getting deep into the innards of the cleaning in the first major step of the project, so gaining some general familiarity can help you get your bearings.

In [2]:
# Load in the general demographics data.
azdias = pd.read_csv('./Udacity_AZDIAS_Subset.csv', delimiter = ';')

# Load in the feature summary file.
feat_info = pd.read_csv('./AZDIAS_Feature_Summary.csv', delimiter = ';')

In [3]:
#the following cell is executed and the final output is saved in the workspace directory.

> **Tip**: Add additional cells to keep everything in reasonably-sized chunks! Keyboard shortcut `esc --> a` (press escape to enter command mode, then press the 'A' key) adds a new cell before the active cell, and `esc --> b` adds a new cell after the active cell. If you need to convert an active cell to a markdown cell, use `esc --> m` and to convert to a code cell, use `esc --> y`. 

## Step 1: Preprocessing

### Step 1.1: Assess Missing Data

The feature summary file contains a summary of properties for each demographics data column. You will use this file to help you make cleaning decisions during this stage of the project. First of all, you should assess the demographics data in terms of missing data. Pay attention to the following points as you perform your analysis, and take notes on what you observe. Make sure that you fill in the **Discussion** cell with your findings and decisions at the end of each step that has one!

#### Step 1.1.1: Convert Missing Value Codes to NaNs
The fourth column of the feature attributes summary (loaded in above as `feat_info`) documents the codes from the data dictionary that indicate missing or unknown data. While the file encodes this as a list (e.g. `[-1,0]`), this will get read in as a string object. You'll need to do a little bit of parsing to make use of it to identify and clean the data. Convert data that matches a 'missing' or 'unknown' value code into a numpy NaN value. You might want to see how much data takes on a 'missing' or 'unknown' code, and how much data is naturally missing, as a point of interest.

**As one more reminder, you are encouraged to add additional cells to break up your analysis into manageable chunks.**

In [4]:
#removing duplicate rows
azdias = azdias.drop_duplicates()
len(azdias)

838727

In [5]:
# Identify missing or unknown data values and convert them to NaNs.
for i in range(azdias.shape[1]):
    column = azdias.columns[i]
    nan = feat_info.iloc[i, 3]
    nan = nan.replace('[','').replace(']','').split(',')
    if nan != ['']:
        for n in nan:
            if azdias[column].dtype == 'object':
                azdias.loc[azdias[column]==n, column] = np.nan
            else:
                n = int(n)
                azdias.loc[azdias[column]==n, column] = np.nan
print('Total number of missing values after conversion to NaNs is: {}'.format(azdias.isnull().sum().sum()))

Total number of missing values after conversion to NaNs is: 5633828


#### Step 1.1.2: Assess Missing Data in Each Column

How much missing data is present in each column? There are a few columns that are outliers in terms of the proportion of values that are missing. You will want to use matplotlib's [`hist()`](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.hist.html) function to visualize the distribution of missing value counts to find these columns. Identify and document these columns. While some of these columns might have justifications for keeping or re-encoding the data, for this project you should just remove them from the dataframe. (Feel free to make remarks about these outlier columns in the discussion, however!)

For the remaining features, are there any patterns in which columns have, or share, missing data?

In [6]:
# Perform an assessment of how much missing data there is in each column of the
# dataset.
nans = azdias.isnull().sum()
nans

AGER_TYP                 633352
ALTERSKATEGORIE_GROB       2881
ANREDE_KZ                     0
CJT_GESAMTTYP              4854
FINANZ_MINIMALIST             0
FINANZ_SPARER                 0
FINANZ_VORSORGER              0
FINANZ_ANLEGER                0
FINANZ_UNAUFFAELLIGER         0
FINANZ_HAUSBAUER              0
FINANZTYP                     0
GEBURTSJAHR              339825
GFK_URLAUBERTYP            4854
GREEN_AVANTGARDE              0
HEALTH_TYP                59086
LP_LEBENSPHASE_FEIN       51846
LP_LEBENSPHASE_GROB       48794
LP_FAMILIE_FEIN           32217
LP_FAMILIE_GROB           32217
LP_STATUS_FEIN             4854
LP_STATUS_GROB             4854
NATIONALITAET_KZ          56205
PRAEGENDE_JUGENDJAHRE     55869
RETOURTYP_BK_S             4854
SEMIO_SOZ                     0
SEMIO_FAM                     0
SEMIO_REL                     0
SEMIO_MAT                     0
SEMIO_VERT                    0
SEMIO_LUST                    0
                          ...  
OST_WEST

# Investigate patterns in the amount of missing data in each column.
fig = go.Figure(data=[go.Histogram(x=nans)])
fig.show()

In [8]:
#detecting outliers
azdias_outliers = azdias.loc[:, (azdias.isnull().sum(axis=0) > 200000)]
for col in azdias_outliers.columns:
    print(col)

AGER_TYP
GEBURTSJAHR
TITEL_KZ
ALTER_HH
KK_KUNDENTYP
KBA05_BAUMAX


In [9]:
azdias_outliers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 838727 entries, 0 to 891220
Data columns (total 6 columns):
AGER_TYP        205375 non-null float64
GEBURTSJAHR     498902 non-null float64
TITEL_KZ        2160 non-null float64
ALTER_HH        580950 non-null float64
KK_KUNDENTYP    306609 non-null float64
KBA05_BAUMAX    414466 non-null float64
dtypes: float64(6)
memory usage: 44.8 MB


# Remove the outlier columns from the dataset. (You'll perform other data
# engineering tasks such as re-encoding and imputation later.)
azdias = azdias.loc[:, (azdias.isnull().sum(axis=0) <= 200000)]
#checking NaNs distribution after filtering
nans_filter = azdias.isnull().sum()
fig = go.Figure(data=[go.Histogram(x=nans_filter)])
fig.show()

#### Discussion 1.1.2: Assess Missing Data in Each Column
These columns have been dropped from azdias dataframe: [AGER_TYP,GEBURTSJAHR,TITEL_KZ,ALTER_HH,KK_KUNDENTYP,KBA05_BAUMAX] as they have outlier ranges from 300k to 900k NaNs. I think they're null due to there reference as personal details that the data entry clerks might not give attention to their status, except for ['KK_KUNDENTYP'].

#### Step 1.1.3: Assess Missing Data in Each Row

Now, you'll perform a similar assessment for the rows of the dataset. How much data is missing in each row? As with the columns, you should see some groups of points that have a very different numbers of missing values. Divide the data into two subsets: one for data points that are above some threshold for missing values, and a second subset for points below that threshold.

In order to know what to do with the outlier rows, we should see if the distribution of data values on columns that are not missing data (or are missing very little data) are similar or different between the two groups. Select at least five of these columns and compare the distribution of values.
- You can use seaborn's [`countplot()`](https://seaborn.pydata.org/generated/seaborn.countplot.html) function to create a bar chart of code frequencies and matplotlib's [`subplot()`](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.subplot.html) function to put bar charts for the two subplots side by side.
- To reduce repeated code, you might want to write a function that can perform this comparison, taking as one of its arguments a column to be compared.

Depending on what you observe in your comparison, this will have implications on how you approach your conclusions later in the analysis. If the distributions of non-missing features look similar between the data with many missing values and the data with few or no missing values, then we could argue that simply dropping those points from the analysis won't present a major issue. On the other hand, if the data with many missing values looks very different from the data with few or no missing values, then we should make a note on those data as special. We'll revisit these data later on. **Either way, you should continue your analysis for now using just the subset of the data with few or no missing values.**

# How much data is missing in each row of the dataset?
nans_rows = azdias.isnull().sum(axis=1)
fig = go.Figure(data=[go.Histogram(x=nans_rows)])
fig.show()

# Write code to divide the data into two subsets based on the number of missing
# values in each row.
azdias_1 = azdias.loc[(azdias.isnull().sum(axis=1) >= 30), :]
azdias_2 = azdias.loc[(azdias.isnull().sum(axis=1) < 30), :]
nans_1 = azdias_1.isnull().sum(axis=1)
nans_2 = azdias_2.isnull().sum(axis=1)

fig = make_subplots(rows=1, cols=2)
fig.add_trace(go.Histogram(x=nans_1, name= ">=30"), row=1, col=1)
fig.add_trace(go.Histogram(x=nans_2, name= "<30"), row=1, col=2)
fig.update_layout(height=600, width=800, title_text="Side By Side Subplots")
fig.show()

In [13]:
print("Percentage of NaNs above 30 is: {}%".format(round(azdias_1.shape[0]/azdias.shape[0]*100,2)))
print("Percentage of NaNs below 30 is: {}%".format(round(azdias_2.shape[0]/azdias.shape[0]*100,2)))

Percentage of NaNs above 30 is: 4.89%
Percentage of NaNs below 30 is: 95.11%


In [14]:
#columns with zero NaN values
zero_nans = []
for i in range(nans.shape[0]):
    if nans[i] == 0:
        zero_nans.append(nans.index[i])
zero_nans

['ANREDE_KZ',
 'FINANZ_MINIMALIST',
 'FINANZ_SPARER',
 'FINANZ_VORSORGER',
 'FINANZ_ANLEGER',
 'FINANZ_UNAUFFAELLIGER',
 'FINANZ_HAUSBAUER',
 'FINANZTYP',
 'GREEN_AVANTGARDE',
 'SEMIO_SOZ',
 'SEMIO_FAM',
 'SEMIO_REL',
 'SEMIO_MAT',
 'SEMIO_VERT',
 'SEMIO_LUST',
 'SEMIO_ERL',
 'SEMIO_KULT',
 'SEMIO_RAT',
 'SEMIO_KRIT',
 'SEMIO_DOM',
 'SEMIO_KAEM',
 'SEMIO_PFLICHT',
 'SEMIO_TRADV',
 'ZABEOTYP']

#### Discussion 1.1.3: Assess Missing Data in Each Row
In accordance with the row analysis and the column-comparison analysis, we ended up with two datasets, the first tells us that only 5% of the whole azdias has rows of more than or equal to 30 NaNs, while the second tells us that 95% of the azdias dataset has rows of less than 30 NaNs which will lead us to drop the 5%-rows with no worry of affecting the further analysis negatively. Also, the column-comparison analysis will lead us to the same result.

### Step 1.2: Select and Re-Encode Features

Checking for missing data isn't the only way in which you can prepare a dataset for analysis. Since the unsupervised learning techniques to be used will only work on data that is encoded numerically, you need to make a few encoding changes or additional assumptions to be able to make progress. In addition, while almost all of the values in the dataset are encoded using numbers, not all of them represent numeric values. Check the third column of the feature summary (`feat_info`) for a summary of types of measurement.
- For numeric and interval data, these features can be kept without changes.
- Most of the variables in the dataset are ordinal in nature. While ordinal values may technically be non-linear in spacing, make the simplifying assumption that the ordinal variables can be treated as being interval in nature (that is, kept without any changes).
- Special handling may be necessary for the remaining two variable types: categorical, and 'mixed'.

In the first two parts of this sub-step, you will perform an investigation of the categorical and mixed-type features and make a decision on each of them, whether you will keep, drop, or re-encode each. Then, in the last part, you will create a new data frame with only the selected and engineered columns.

Data wrangling is often the trickiest part of the data analysis process, and there's a lot of it to be done here. But stick with it: once you're done with this step, you'll be ready to get to the machine learning parts of the project!

In [15]:
# How many features are there of each data type?
azdias = azdias_2 #converting to deal with the subset that has fewer or no NaNs
feat_info = feat_info[feat_info.attribute.isin(list(azdias.columns.values))]
feat_info['type'].value_counts()

ordinal        49
categorical    18
mixed           6
numeric         6
Name: type, dtype: int64

#### Step 1.2.1: Re-Encode Categorical Features

For categorical data, you would ordinarily need to encode the levels as dummy variables. Depending on the number of categories, perform one of the following:
- For binary (two-level) categoricals that take numeric values, you can keep them without needing to do anything.
- There is one binary variable that takes on non-numeric values. For this one, you need to re-encode the values as numbers or create a dummy variable.
- For multi-level categoricals (three or more values), you can choose to encode the values using multiple dummy variables (e.g. via [OneHotEncoder](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)), or (to keep things straightforward) just drop them from the analysis. As always, document your choices in the Discussion section.

In [16]:
# Assess categorical variables: which are binary, which are multi-level, and
# which one needs to be re-encoded?
binary = []
multi_level = []
azdias_cat = feat_info[feat_info.type=='categorical']
for col in azdias_cat.attribute.values:
    if azdias[col].nunique() <= 2:
        binary.append(col)
    else:
        multi_level.append(col)
print("binary attributes: {}".format(binary))
print("====================================")
print("multi_level attributes: {}".format(multi_level))

binary attributes: ['ANREDE_KZ', 'GREEN_AVANTGARDE', 'SOHO_KZ', 'VERS_TYP', 'OST_WEST_KZ']
multi_level attributes: ['CJT_GESAMTTYP', 'FINANZTYP', 'GFK_URLAUBERTYP', 'LP_FAMILIE_FEIN', 'LP_FAMILIE_GROB', 'LP_STATUS_FEIN', 'LP_STATUS_GROB', 'NATIONALITAET_KZ', 'SHOPPER_TYP', 'ZABEOTYP', 'GEBAEUDETYP', 'CAMEO_DEUG_2015', 'CAMEO_DEU_2015']


In [17]:
# Re-encode categorical variable(s) to be kept in the analysis.
#dropping multi_level columns
azdias_clean = azdias.drop(multi_level, axis=1)
#re-enocding bianry columns
azdias_clean = pd.get_dummies(azdias_clean, columns=binary)

#### Discussion 1.2.1: Re-Encode Categorical Features
Categorical features are divided into binary and multi_level features, the latter is dropped and the binary features are re-encoded with pd.get_dummies() to standarize the codes

#### Step 1.2.2: Engineer Mixed-Type Features

There are a handful of features that are marked as "mixed" in the feature summary that require special treatment in order to be included in the analysis. There are two in particular that deserve attention; the handling of the rest are up to your own choices:
- "PRAEGENDE_JUGENDJAHRE" combines information on three dimensions: generation by decade, movement (mainstream vs. avantgarde), and nation (east vs. west). While there aren't enough levels to disentangle east from west, you should create two new variables to capture the other two dimensions: an interval-type variable for decade, and a binary variable for movement.
- "CAMEO_INTL_2015" combines information on two axes: wealth and life stage. Break up the two-digit codes by their 'tens'-place and 'ones'-place digits into two new ordinal variables (which, for the purposes of this project, is equivalent to just treating them as their raw numeric values).
- If you decide to keep or engineer new features around the other mixed-type features, make sure you note your steps in the Discussion section.

Be sure to check `Data_Dictionary.md` for the details needed to finish these tasks.

In [18]:
azdias_mixed = feat_info[feat_info.type=='mixed']
azdias_mixed

,attribute,information_level,type,missing_or_unknown
15,LP_LEBENSPHASE_FEIN,person,mixed,[0]
16,LP_LEBENSPHASE_GROB,person,mixed,[0]
22,PRAEGENDE_JUGENDJAHRE,person,mixed,"[-1,0]"
56,WOHNLAGE,building,mixed,[-1]
59,CAMEO_INTL_2015,microcell_rr4,mixed,"[-1,XX]"
79,PLZ8_BAUMAX,macrocell_plz8,mixed,"[-1,0]"


In [19]:
#investigating data types in each column
print(azdias_clean["PRAEGENDE_JUGENDJAHRE"].dtype)
print(azdias_clean["CAMEO_INTL_2015"].dtype)

float64
object


In [20]:
# Investigate "PRAEGENDE_JUGENDJAHRE" and engineer two new variables.
#mapping "PRAEGENDE_JUGENDJAHRE" with dictionaries
decade = {1:1, 2:1, 3:2, 4:2, 5:3, 6:3, 7:3, 8:4, 9:4, 10:5, 11:5, 12:5, 13:5, 14:6, 15:6}
movement = {1:1, 2:0, 3:1, 4:0, 5:1, 6:0, 7:0, 8:1, 9:0, 10:1, 11:0, 12:1, 13:0, 14:1, 15:0}
azdias_clean['DECADE'] = azdias_clean["PRAEGENDE_JUGENDJAHRE"].map(decade)
azdias_clean['MOVEMENT'] = azdias_clean["PRAEGENDE_JUGENDJAHRE"].map(movement)

In [21]:
# Investigate "CAMEO_INTL_2015" and engineer two new variables.
wealth = {'11':1, '12':1, '13':1, '14':1, '15':1, '21':2, '22':2, '23':2, '24':2, '25':2,
          '31':3, '32':3, '33':3, '34':3, '35':3, '41':4, '42':4, '43':4, '44':4, '45':4,
          '51':5, '52':5, '53':5, '54':5, '55':5}
life_stage = {'11':1, '12':2, '13':3, '14':4, '15':5, '21':1, '22':2, '23':3, '24':4, '25':5,
              '31':1, '32':2, '33':3, '34':4, '35':5, '41':1, '42':2, '43':3, '44':4, '45':5,
              '51':1, '52':2, '53':3, '54':4, '55':5}
azdias_clean['WEALTH'] = azdias_clean["CAMEO_INTL_2015"].map(wealth)
azdias_clean['LIFE_STAGE'] = azdias_clean["CAMEO_INTL_2015"].map(life_stage)
azdias_clean = azdias_clean.drop(["PRAEGENDE_JUGENDJAHRE", "CAMEO_INTL_2015"], axis=1) 
azdias_clean.shape

(797723, 73)

#### Discussion 1.2.2: Engineer Mixed-Type Features
To perfectly deal with "PRAEGENDE_JUGENDJAHRE" and "CAMEO_INTL_2015", a lookup to Data_Dictionary.md was made to determine how to create new two variables per initial variable to be able to encode them as numerical values, so two dictionaries per variable are initialized and then mapped with their original variable to fill in the new variables with encoded values.

#### Step 1.2.3: Complete Feature Selection

In order to finish this step up, you need to make sure that your data frame now only has the columns that you want to keep. To summarize, the dataframe should consist of the following:
- All numeric, interval, and ordinal type columns from the original dataset.
- Binary categorical features (all numerically-encoded).
- Engineered features from other multi-level categorical features and mixed features.

Make sure that for any new columns that you have engineered, that you've excluded the original columns from the final dataset. Otherwise, their values will interfere with the analysis later on the project. For example, you should not keep "PRAEGENDE_JUGENDJAHRE", since its values won't be useful for the algorithm: only the values derived from it in the engineered features you created should be retained. As a reminder, your data should only be from **the subset with few or no missing values**.

In [22]:
#checking for NaNs
azdias_clean.isnull().sum().sum()

846807

In [23]:
#filling NaNs with most frequent values
azdias_clean = azdias_clean.fillna(azdias_clean.apply(lambda x: x.value_counts().index[0]))
azdias_clean.isnull().sum().sum()

0

### Step 1.3: Create a Cleaning Function

Even though you've finished cleaning up the general population demographics data, it's important to look ahead to the future and realize that you'll need to perform the same cleaning steps on the customer demographics data. In this substep, complete the function below to execute the main feature selection, encoding, and re-engineering steps you performed above. Then, when it comes to looking at the customer data in Step 3, you can just run this function on that DataFrame to get the trimmed dataset in a single step.

In [24]:
def clean_data(df):
    """
    Perform feature trimming, re-encoding, and engineering for demographics
    data
    
    INPUT: Demographics DataFrame
    OUTPUT: Trimmed and cleaned demographics DataFrame
    """
    #reading summary file
    feat_info = pd.read_csv('./AZDIAS_Feature_Summary.csv', delimiter = ';')
    #performing basic cleaning
    for i in range(df.shape[1]):
        column = df.columns[i]
        nan = feat_info.iloc[i, 3]
        nan = nan.replace('[','').replace(']','').split(',')
        if nan != ['']:
            for n in nan:
                if df[column].dtype == 'object':
                    df.loc[df[column]==n, column] = np.nan
                else:
                    n = int(n)
                    df.loc[df[column]==n, column] = np.nan
                
    #converting to data frame with less number of NaNs across columns and rows           
    df = df.loc[(df.isnull().sum(axis=1) < 30), (df.isnull().sum(axis=0) <= 200000)]
    feat_info = feat_info[feat_info.attribute.isin(list(df.columns.values))]
    
    #re-enocding categorical bianry columns and dropping multi_level categorical columns
    binary = []
    multi_level = []
    df_cat = feat_info[feat_info.type=='categorical']
    for col in df_cat.attribute.values:
        if df[col].nunique() <= 2:
            binary.append(col)
        else:
            multi_level.append(col)
    for i in multi_level:
        df = df.drop(i, axis=1)
    df = pd.get_dummies(df, columns=binary)
    
    #mapping new variables with dictionaries from df["PRAEGENDE_JUGENDJAHRE"]'s and df["CAMEO_INTL_2015"]'s values
    decade = {1:1, 2:1, 3:2, 4:2, 5:3, 6:3, 7:3, 8:4, 9:4, 10:5, 11:5, 12:5, 13:5, 14:6, 15:6}
    movement = {1:1, 2:0, 3:1, 4:0, 5:1, 6:0, 7:0, 8:1, 9:0, 10:1, 11:0, 12:1, 13:0, 14:1, 15:0}
    wealth = {'11':1, '12':1, '13':1, '14':1, '15':1, '21':2, '22':2, '23':2, '24':2, '25':2,
              '31':3, '32':3, '33':3, '34':3, '35':3, '41':4, '42':4, '43':4, '44':4, '45':4,
              '51':5, '52':5, '53':5, '54':5, '55':5}
    life_stage = {'11':1, '12':2, '13':3, '14':4, '15':5, '21':1, '22':2, '23':3, '24':4, '25':5,
                  '31':1, '32':2, '33':3, '34':4, '35':5, '41':1, '42':2, '43':3, '44':4, '45':5,
                  '51':1, '52':2, '53':3, '54':4, '55':5}
    df['DECADE'] = df["PRAEGENDE_JUGENDJAHRE"].map(decade)
    df['MOVEMENT'] = df["PRAEGENDE_JUGENDJAHRE"].map(movement)
    df['WEALTH'] = df["CAMEO_INTL_2015"].map(wealth)
    df['LIFE_STAGE'] = df["CAMEO_INTL_2015"].map(life_stage)
    df = df.drop(["PRAEGENDE_JUGENDJAHRE", "CAMEO_INTL_2015"], axis=1) #dropping no-need variables
    df = df.fillna(df.apply(lambda x: x.value_counts().index[0])) #filling NaNs with the most frequent values
    return df

In [25]:
#making sure that the function is working properly
df = pd.read_csv('./Udacity_AZDIAS_Subset.csv', delimiter = ';')
df = clean_data(df)
print("After executing the clean_df function, the number of NaNs is: {}".format(df.isnull().sum().sum()))

After executing the clean_df function, the number of NaNs is: 0


## Step 2: Feature Transformation

### Step 2.1: Apply Feature Scaling

Before we apply dimensionality reduction techniques to the data, we need to perform feature scaling so that the principal component vectors are not influenced by the natural differences in scale for features. Starting from this part of the project, you'll want to keep an eye on the [API reference page for sklearn](http://scikit-learn.org/stable/modules/classes.html) to help you navigate to all of the classes and functions that you'll need. In this substep, you'll need to check the following:

- sklearn requires that data not have missing values in order for its estimators to work properly. So, before applying the scaler to your data, make sure that you've cleaned the DataFrame of the remaining missing values. This can be as simple as just removing all data points with missing data, or applying an [Imputer](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Imputer.html) to replace all missing values. You might also try a more complicated procedure where you temporarily remove missing values in order to compute the scaling parameters before re-introducing those missing values and applying imputation. Think about how much missing data you have and what possible effects each approach might have on your analysis, and justify your decision in the discussion section below.
- For the actual scaling function, a [StandardScaler](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) instance is suggested, scaling each feature to mean 0 and standard deviation 1.
- For these classes, you can make use of the `.fit_transform()` method to both fit a procedure to the data as well as apply the transformation to the data at the same time. Don't forget to keep the fit sklearn objects handy, since you'll be applying them to the customer demographics data towards the end of the project.

In [26]:
# Apply feature scaling to the general population demographics data.
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
azdias_transform = scaler.fit_transform(azdias_clean)

### Discussion 2.1: Apply Feature Scaling
Dealing with NaNs and applying feature scaling are done thorugh:
* filling all NaNs with the most frequent values in each column.
* applying StandardScaler to scale the feature.

### Step 2.2: Perform Dimensionality Reduction

On your scaled data, you are now ready to apply dimensionality reduction techniques.

- Use sklearn's [PCA](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) class to apply principal component analysis on the data, thus finding the vectors of maximal variance in the data. To start, you should not set any parameters (so all components are computed) or set a number of components that is at least half the number of features (so there's enough features to see the general trend in variability).
- Check out the ratio of variance explained by each principal component as well as the cumulative variance explained. Try plotting the cumulative or sequential values using matplotlib's [`plot()`](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.plot.html) function. Based on what you find, select a value for the number of transformed features you'll retain for the clustering part of the project.
- Once you've made a choice for the number of components to keep, make sure you re-fit a PCA instance to perform the decided-on transformation.

In [27]:
# Apply PCA to the data.
from sklearn.decomposition import PCA
pca = PCA()
pca_fit = pca.fit(azdias_transform)

# Investigate the variance accounted for by each principal component.
rve = make_subplots(rows=1, cols=2)
rve.add_trace(go.Bar(x=list(range(len(pca_fit.explained_variance_ratio_))),
                     y=pca_fit.explained_variance_ratio_.tolist(),
                     name= "Explained Variance Ratio"),row=1, col=1)
rve.add_trace(go.Scatter(x=list(range(len(pca_fit.explained_variance_ratio_))),
                         y=np.cumsum(pca_fit.explained_variance_ratio_.tolist()),
                         name="Cummulative Variance Explained",
                         mode='lines+markers'), row=1, col=2)

rve.update_layout(height=600, width=1000, title_text="Variance Explanation")
rve.show()

In [29]:
# Re-apply PCA to the data while selecting for number of components to retain.
pca_36 = PCA(n_components=36)
pca_azdias = pca_36.fit_transform(azdias_transform)

### Discussion 2.2: Perform Dimensionality Reduction
Although the cummulative explained varience ratio for the whole features is the almost the same for n_components=36, but I'll choose to reduce the diemnsions of the dataset

### Step 2.3: Interpret Principal Components

Now that we have our transformed principal components, it's a nice idea to check out the weight of each variable on the first few components to see if they can be interpreted in some fashion.

As a reminder, each principal component is a unit vector that points in the direction of highest variance (after accounting for the variance captured by earlier principal components). The further a weight is from zero, the more the principal component is in the direction of the corresponding feature. If two features have large weights of the same sign (both positive or both negative), then increases in one tend expect to be associated with increases in the other. To contrast, features with different signs can be expected to show a negative correlation: increases in one variable should result in a decrease in the other.

- To investigate the features, you should map each weight to their corresponding feature name, then sort the features according to weight. The most interesting features for each principal component, then, will be those at the beginning and end of the sorted list. Use the data dictionary document to help you understand these most prominent features, their relationships, and what a positive or negative value on the principal component might indicate.
- You should investigate and interpret feature associations from the first three principal components in this substep. To help facilitate this, you should write a function that you can call at any time to print the sorted list of feature weights, for the *i*-th principal component. This might come in handy in the next step of the project, when you interpret the tendencies of the discovered clusters.

In [30]:
# Map weights for the first principal component to corresponding feature names
# and then print the linked values, sorted by weight.
# HINT: Try defining a function here or in a new cell that you can reuse in the
# other cells.
def pca_weights(pca_weights, i):
    df = pd.DataFrame(pca_weights.components_, columns=list(azdias_clean.columns))
    weights = df.iloc[i].sort_values(ascending=False)
    return weights
first_component = pca_weights(pca_36, 0)
first_component

PLZ8_ANTG3               0.220266
PLZ8_ANTG4               0.213971
PLZ8_BAUMAX              0.211355
HH_EINKOMMEN_SCORE       0.199816
WEALTH                   0.199697
ORTSGR_KLS9              0.190779
EWDICHTE                 0.187794
FINANZ_HAUSBAUER         0.164845
KBA05_ANTG4              0.151540
PLZ8_ANTG2               0.148274
ARBEIT                   0.140287
ANZ_HAUSHALTE_AKTIV      0.134071
KBA05_ANTG3              0.132952
RELAT_AB                 0.130002
FINANZ_SPARER            0.119008
GREEN_AVANTGARDE_0       0.115242
MOVEMENT                 0.115242
SEMIO_PFLICHT            0.089128
SEMIO_REL                0.085063
DECADE                   0.079233
SEMIO_RAT                0.074202
SEMIO_TRADV              0.063161
SEMIO_MAT                0.060378
OST_WEST_KZ_O            0.060083
W_KEIT_KIND_HH           0.058346
SEMIO_FAM                0.058071
REGIOTYP                 0.057690
FINANZ_UNAUFFAELLIGER    0.050701
FINANZ_ANLEGER           0.050400
SEMIO_KULT    

In [31]:
# Map weights for the second principal component to corresponding feature names
# and then print the linked values, sorted by weight.
second_component = pca_weights(pca_36, 1)
second_component

ALTERSKATEGORIE_GROB     0.257806
SEMIO_ERL                0.250776
FINANZ_VORSORGER         0.229655
SEMIO_LUST               0.179560
RETOURTYP_BK_S           0.149917
SEMIO_KRIT               0.140529
SEMIO_KAEM               0.140244
ANREDE_KZ_2.0            0.125940
W_KEIT_KIND_HH           0.101393
FINANZ_HAUSBAUER         0.100463
SEMIO_DOM                0.099734
FINANZ_MINIMALIST        0.074556
EWDICHTE                 0.064817
ORTSGR_KLS9              0.063783
WOHNDAUER_2008           0.061557
PLZ8_ANTG3               0.060613
PLZ8_ANTG4               0.060079
PLZ8_BAUMAX              0.055196
KBA05_ANTG4              0.048130
ARBEIT                   0.047624
RELAT_AB                 0.045459
WEALTH                   0.044221
ANZ_HAUSHALTE_AKTIV      0.042341
PLZ8_ANTG2               0.041905
VERS_TYP_2.0             0.032976
HH_EINKOMMEN_SCORE       0.031820
KBA05_ANTG3              0.028220
ANZ_HH_TITEL             0.024982
OST_WEST_KZ_O            0.019868
LIFE_STAGE    

In [32]:
# Map weights for the third principal component to corresponding feature names
# and then print the linked values, sorted by weight.
third_component = pca_weights(pca_36, 2)
third_component

ANREDE_KZ_1.0            0.334820
SEMIO_VERT               0.312854
SEMIO_SOZ                0.228136
SEMIO_FAM                0.204770
SEMIO_KULT               0.188073
FINANZ_MINIMALIST        0.157092
FINANZ_VORSORGER         0.122487
RETOURTYP_BK_S           0.120516
ALTERSKATEGORIE_GROB     0.110157
W_KEIT_KIND_HH           0.089095
SEMIO_LUST               0.083240
ORTSGR_KLS9              0.059648
EWDICHTE                 0.058415
PLZ8_BAUMAX              0.056416
PLZ8_ANTG4               0.055304
PLZ8_ANTG3               0.054172
GREEN_AVANTGARDE_1       0.052753
ARBEIT                   0.042655
RELAT_AB                 0.039074
WOHNDAUER_2008           0.037873
PLZ8_ANTG2               0.036358
KBA05_ANTG4              0.032970
VERS_TYP_2.0             0.032168
WEALTH                   0.031007
ANZ_HAUSHALTE_AKTIV      0.029153
SEMIO_REL                0.025857
SEMIO_MAT                0.025724
OST_WEST_KZ_O            0.018636
ANZ_HH_TITEL             0.016101
KBA05_ANTG3   

### Discussion 2.3: Interpret Principal Components
From the first principal component, it can be seen that

- PLZ8_ANTG3 and PLZ8_ANTG4 have a positive correlation. The share of 10+ family homes and the share of 6-10 family homes for macro-cells tend to increase together.
- PLZ8_ANTG3 and WEALTH also have a positive correlation. It indicates that when the share of 6-10 family homes in a macro-cell increases, it tends to have more poor households.
- MOBI_REGIO and PLZ8_ANTG3 have a negative correlation. It means that when the share of 6-10 family homes in a macro-cell increases, it tends to have higher movements.

## Step 3: Clustering

### Step 3.1: Apply Clustering to General Population

You've assessed and cleaned the demographics data, then scaled and transformed them. Now, it's time to see how the data clusters in the principal components space. In this substep, you will apply k-means clustering to the dataset and use the average within-cluster distances from each point to their assigned cluster's centroid to decide on a number of clusters to keep.

- Use sklearn's [KMeans](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans) class to perform k-means clustering on the PCA-transformed data.
- Then, compute the average difference from each point to its assigned cluster's center. **Hint**: The KMeans object's `.score()` method might be useful here, but note that in sklearn, scores tend to be defined so that larger is better. Try applying it to a small, toy dataset, or use an internet search to help your understanding.
- Perform the above two steps for a number of different cluster counts. You can then see how the average distance decreases with an increasing number of clusters. However, each additional cluster provides a smaller net benefit. Use this fact to select a final number of clusters in which to group the data. **Warning**: because of the large size of the dataset, it can take a long time for the algorithm to resolve. The more clusters to fit, the longer the algorithm will take. You should test for cluster counts through at least 10 clusters to get the full picture, but you shouldn't need to test for a number of clusters above about 30.
- Once you've selected a final number of clusters to use, re-fit a KMeans instance to perform the clustering operation. Make sure that you also obtain the cluster assignments for the general demographics data, since you'll be using them in the final Step 3.3.

In [33]:
from sklearn.cluster import KMeans
def kmeans_clustering(df, n_clusters):
    kmeans = KMeans(n_clusters=n_clusters)
    kmeans_fit = kmeans.fit(df)
    score = np.abs(kmeans_fit.score(df))
    return score

In [34]:
ls_score = []
clusters = list(range(5,25,5))
for i in clusters:
    ls_score.append(kmeans_clustering(pca_azdias, i))

In [35]:
# Investigate the change in within-cluster distance across number of clusters.
# HINT: Use matplotlib's plot function to visualize this relationship.
fig = go.Figure(go.Scatter(x=clusters, y=ls_score, mode='lines+markers', name='number_of_clusters'),
               go.Layout(title='Elbow Plot'))
fig.show()

In [36]:
# Re-fit the k-means model with the selected number of clusters and obtain
# cluster predictions for the general population demographics data.



### Discussion 3.1: Apply Clustering to General Population

(Double-click this cell and replace this text with your own text, reporting your findings and decisions regarding clustering. Into how many clusters have you decided to segment the population?)

### Step 3.2: Apply All Steps to the Customer Data

Now that you have clusters and cluster centers for the general population, it's time to see how the customer data maps on to those clusters. Take care to not confuse this for re-fitting all of the models to the customer data. Instead, you're going to use the fits from the general population to clean, transform, and cluster the customer data. In the last step of the project, you will interpret how the general population fits apply to the customer data.

- Don't forget when loading in the customers data, that it is semicolon (`;`) delimited.
- Apply the same feature wrangling, selection, and engineering steps to the customer demographics using the `clean_data()` function you created earlier. (You can assume that the customer demographics data has similar meaning behind missing data patterns as the general demographics data.)
- Use the sklearn objects from the general demographics data, and apply their transformations to the customers data. That is, you should not be using a `.fit()` or `.fit_transform()` method to re-fit the old objects, nor should you be creating new sklearn objects! Carry the data through the feature scaling, PCA, and clustering steps, obtaining cluster assignments for all of the data in the customer demographics data.

In [37]:
# Load in the customer demographics data.
customers = 

SyntaxError: invalid syntax (<ipython-input-37-416d584d1460>, line 2)

In [ ]:
# Apply preprocessing, feature transformation, and clustering from the general
# demographics onto the customer data, obtaining cluster predictions for the
# customer demographics data.



### Step 3.3: Compare Customer Data to Demographics Data

At this point, you have clustered data based on demographics of the general population of Germany, and seen how the customer data for a mail-order sales company maps onto those demographic clusters. In this final substep, you will compare the two cluster distributions to see where the strongest customer base for the company is.

Consider the proportion of persons in each cluster for the general population, and the proportions for the customers. If we think the company's customer base to be universal, then the cluster assignment proportions should be fairly similar between the two. If there are only particular segments of the population that are interested in the company's products, then we should see a mismatch from one to the other. If there is a higher proportion of persons in a cluster for the customer data compared to the general population (e.g. 5% of persons are assigned to a cluster for the general population, but 15% of the customer data is closest to that cluster's centroid) then that suggests the people in that cluster to be a target audience for the company. On the other hand, the proportion of the data in a cluster being larger in the general population than the customer data (e.g. only 2% of customers closest to a population centroid that captures 6% of the data) suggests that group of persons to be outside of the target demographics.

Take a look at the following points in this step:

- Compute the proportion of data points in each cluster for the general population and the customer data. Visualizations will be useful here: both for the individual dataset proportions, but also to visualize the ratios in cluster representation between groups. Seaborn's [`countplot()`](https://seaborn.pydata.org/generated/seaborn.countplot.html) or [`barplot()`](https://seaborn.pydata.org/generated/seaborn.barplot.html) function could be handy.
  - Recall the analysis you performed in step 1.1.3 of the project, where you separated out certain data points from the dataset if they had more than a specified threshold of missing values. If you found that this group was qualitatively different from the main bulk of the data, you should treat this as an additional data cluster in this analysis. Make sure that you account for the number of data points in this subset, for both the general population and customer datasets, when making your computations!
- Which cluster or clusters are overrepresented in the customer dataset compared to the general population? Select at least one such cluster and infer what kind of people might be represented by that cluster. Use the principal component interpretations from step 2.3 or look at additional components to help you make this inference. Alternatively, you can use the `.inverse_transform()` method of the PCA and StandardScaler objects to transform centroids back to the original data space and interpret the retrieved values directly.
- Perform a similar investigation for the underrepresented clusters. Which cluster or clusters are underrepresented in the customer dataset compared to the general population, and what kinds of people are typified by these clusters?

In [ ]:
# Compare the proportion of data in each cluster for the customer data to the
# proportion of data in each cluster for the general population.



In [ ]:
# What kinds of people are part of a cluster that is overrepresented in the
# customer data compared to the general population?



In [ ]:
# What kinds of people are part of a cluster that is underrepresented in the
# customer data compared to the general population?



### Discussion 3.3: Compare Customer Data to Demographics Data

(Double-click this cell and replace this text with your own text, reporting findings and conclusions from the clustering analysis. Can we describe segments of the population that are relatively popular with the mail-order company, or relatively unpopular with the company?)

> Congratulations on making it this far in the project! Before you finish, make sure to check through the entire notebook from top to bottom to make sure that your analysis follows a logical flow and all of your findings are documented in **Discussion** cells. Once you've checked over all of your work, you should export the notebook as an HTML document to submit for evaluation. You can do this from the menu, navigating to **File -> Download as -> HTML (.html)**. You will submit both that document and this notebook for your project submission.